In [1]:
import numpy as np
import scann

2023-10-18 20:17:47.872718: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-10-18 20:17:47.909078: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-10-18 20:17:47.910099: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-18 20:17:48.578947: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
v1 = np.array([
        np.array([
           np.random.normal(1, 0.01),
           np.random.normal(0, 0.01),
           np.random.normal(0, 0.01)
        ])
        for _ in range(1000000)
     ]).astype(np.float32)

v2 = np.array([
        np.array([
           np.random.normal(0, 0.01),
           np.random.normal(1, 0.01),
           np.random.normal(0, 0.01)
        ])
        for _ in range(1000000)
     ]).astype(np.float32)

v3 = np.array([
        np.array([
           np.random.normal(0, 0.01),
           np.random.normal(0, 0.01),
           np.random.normal(1, 0.01)
        ])
        for _ in range(1000000)
     ]).astype(np.float32)
    

In [13]:
needle = np.array([np.array([
     np.random.normal( 50, 1),
     np.random.normal( 5, 1),
     np.random.normal( 5, 1)
])])

In [32]:
needles = np.array([
       np.array([
          np.random.normal(0.5, 0.01),
          np.random.normal(0.5, 0.01),
          np.random.normal(0.5, 0.01)
     ])
     for _ in range(10000)
   ]).astype(np.float32)

In [43]:
v1

array([[ 1.0048490e+00,  5.4896348e-03, -2.5233938e-04],
       [ 9.9792808e-01, -1.4496710e-02, -1.7778460e-02],
       [ 1.0066119e+00,  1.2334182e-03, -1.1015638e-02],
       ...,
       [ 9.8896122e-01,  1.4003148e-02, -1.7085336e-02],
       [ 1.0089171e+00,  5.8191470e-03,  1.0911878e-02],
       [ 1.0004807e+00,  4.2558564e-03,  4.3928805e-03]], dtype=float32)

In [44]:
needles

array([[0.50668997, 0.49476546, 0.4976623 ],
       [0.5019856 , 0.49950767, 0.4896393 ],
       [0.478739  , 0.5125522 , 0.50982255],
       [0.5002537 , 0.50475055, 0.49833453],
       [0.4955774 , 0.5012077 , 0.5020446 ],
       [0.5006375 , 0.5158877 , 0.5126605 ],
       [0.4993733 , 0.5067963 , 0.5067914 ],
       [0.50668764, 0.5216731 , 0.48337948],
       [0.48838416, 0.50161755, 0.51198953],
       [0.5077985 , 0.5093175 , 0.49403498],
       [0.5093168 , 0.49968508, 0.5044789 ],
       [0.4954973 , 0.49925476, 0.5039986 ],
       [0.50324386, 0.49592525, 0.5170309 ],
       [0.50945413, 0.49632406, 0.4999179 ],
       [0.4988666 , 0.5184686 , 0.51286435],
       [0.49443594, 0.48943165, 0.4990468 ],
       [0.4979467 , 0.50186145, 0.49763203],
       [0.49115303, 0.5038771 , 0.51311314],
       [0.49943948, 0.48016524, 0.51119214],
       [0.49474666, 0.5011874 , 0.4982313 ],
       [0.5044573 , 0.5016428 , 0.47942066],
       [0.48673788, 0.49796268, 0.49918395],
       [0.

In [33]:
data = np.concatenate( (v1, v2, v3, needles) )

In [51]:
np.random.shuffle(data)

In [34]:
builder = scann.scann_ops_pybind.builder(
    data,                             # numpy array containing the data points to be indexed
    num_neighbors        =  10,              # the number of neighbors to search for
    distance_measure     = "squared_l2"      # the distance metric to use (e.g., 'dot_product' or 'squared_l2')
)

In [63]:
type(builder) # scann.scann_ops.py.scann_builder.ScannBuilder

scann.scann_ops.py.scann_builder.ScannBuilder

In [35]:
builder = builder.tree(                                  # .tree() is a method that can be used after(?) calling .build() to further refine the tree structure of the index. .tree() allows us to fine-tune the tree structure of the index to optimize the search performance for a given dataset and use case. It takes several arguments: 
    num_leaves           =   10000,          # num_leaves: the maximum number of leaves in the tree . This can be used to control the balance between search speed and memory usage. Increasing num_leaves will improve the search accuracy but also increase the memory usage.
    num_leaves_to_search =   10000,          #num_leaves_to_search: the number of leaves to search during the query phase
    training_sample_size = 1000000           # training_sample_size: the number of vectors to use for training the tree.
)

In [29]:
type(builder) # scann.scann_ops.py.scann_builder.ScannBuilder

scann.scann_ops.py.scann_builder.ScannBuilder

In [36]:
builder = builder.score_ah(                              # score_ah() is a method in ScaNN that sets the parameters for the asymmetric hash function used in the indexing process. It is called after .tree() because the tree structure is required to set these parameters.
    10000,                                   # num_neighbors sets the number of neighbors to consider when selecting the threshold for asymmetric hashing   
    anisotropic_quantization_threshold = 0.001 # anisotropic_quantization_threshold controls how many different quantization levels to use for each dimension in the hash function (maller values generally resulting in better accuracy at the cost of higher indexing time and memory usage.)
)

In [32]:
type(builder) # scann.scann_ops.py.scann_builder.ScannBuilder

scann.scann_ops.py.scann_builder.ScannBuilder

In [37]:
builder = builder.reorder(
    1000
)

In [35]:
type(builder) # scann.scann_ops.py.scann_builder.ScannBuilder

scann.scann_ops.py.scann_builder.ScannBuilder

In [38]:
searcher = builder.build()

2023-10-18 20:22:38.909349: I scann/partitioning/partitioner_factory_base.cc:59] Size of sampled dataset for training partition: 999787
2023-10-18 20:22:59.349839: I ./scann/partitioning/kmeans_tree_partitioner_utils.h:84] PartitionerFactory ran in 20.440383133s.


In [15]:
type(searcher) # scann.scann_ops.py.scann_ops_pybind.ScannSearcher

scann.scann_ops.py.scann_ops_pybind.ScannSearcher

In [16]:
tq84_reflect(searcher)

`__class__` ☰ ?
`__delattr__` ☰ ?
`__dict__` ☰ `dict` object
`__dir__()` ☰ Built-in function
`__doc__` ☰ `str` object
`__eq__` ☰ ?
`__format__()` ☰ Built-in function
`__ge__` ☰ ?
`__getattribute__` ☰ ?
`__getstate__()` ☰ Built-in function
`__gt__` ☰ ?
`__hash__` ☰ ?
`__init__()` ☰ Class method
`__init_subclass__()` ☰ Built-in function
`__le__` ☰ ?
`__lt__` ☰ ?
`__module__` ☰ `str` object
`__ne__` ☰ ?
`__new__()` ☰ Built-in function
`__reduce__()` ☰ Built-in function
`__reduce_ex__()` ☰ Built-in function
`__repr__` ☰ ?
`search()` ☰ Class method
`search_batched()` ☰ Class method
`search_batched_parallel()` ☰ Class method
`searcher` ☰ ?
`serialize()` ☰ Class method
`__setattr__` ☰ ?
`__sizeof__()` ☰ Built-in function
`__str__` ☰ ?
`__subclasshook__()` ☰ Built-in function
`__weakref__` ☰ `NoneType` object


In [39]:
query = np.array([ 0.5, 0.5, 0.5 ]).astype(np.float32)

In [40]:
# neighbours is an array of indexes for nearest neighbors. distances is an array with their corresponding squared_L2 distances
neighbors, distances = searcher.search(query, final_num_neighbors=10)

In [13]:
len(neighbors)

10

In [41]:
for x in zip(neighbors,distances): print(x)

(3000898, 2.987506e-06)
(3000494, 5.527281e-06)
(3000081, 7.2080397e-06)
(3000750, 7.3085826e-06)
(3000938, 7.857942e-06)
(3000435, 7.945898e-06)
(3000149, 8.9053265e-06)
(3000821, 9.354817e-06)
(3000100, 1.0098625e-05)
(3000381, 1.0671137e-05)
